# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kasane,-17.8167,25.1500,23.21,17,0,3.26,BW,1695846255
1,1,maloye ganyushkino,46.6000,49.2667,20.98,40,4,2.48,KZ,1695846256
2,2,beaufort west,-32.3567,22.5830,14.21,59,0,4.65,ZA,1695846256
3,3,agat village,13.3886,144.6585,26.52,92,40,0.00,GU,1695846256
4,4,berezna,51.5716,31.7846,13.39,74,8,3.28,UA,1695846256


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
            'Lng',
            'Lat',
            geo = True,
            tiles = 'OSM',
            size = 'Humidity',
            frame_width = 700,
            frame_height = 500,
            color = 'City',
            alpha = 0.5)

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
# ideal weather conditions: 21 < max temp < 27; wind speed < 4.5m/s; zero cloudiness

ideal_cities = city_data_df.loc[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] > 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0),:]

# Drop any rows with null values
ideal_cities.dropna(how = 'any')

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,gadani,25.1185,66.7299,29.12,81,0,2.81,PK,1695846266
91,91,bakewar,26.6667,79.1833,27.55,60,0,1.57,IN,1695846268
350,350,meadow lake,34.8014,-106.5436,30.44,21,0,3.91,US,1695846315
393,393,fort stockton,30.8940,-102.8793,34.53,24,0,2.06,US,1695846320
421,421,crane,31.3974,-102.3501,33.31,24,0,3.40,US,1695846323
455,455,ad dilam,23.9915,47.1627,32.23,17,0,2.45,SA,1695846328
462,462,san miguel,13.4833,-88.1833,34.81,49,0,4.12,SV,1695846251
476,476,lachi,33.3833,71.3386,27.47,40,0,0.78,PK,1695846331


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
79,gadani,PK,25.1185,66.7299,81,
91,bakewar,IN,26.6667,79.1833,60,
350,meadow lake,US,34.8014,-106.5436,21,
393,fort stockton,US,30.8940,-102.8793,24,
421,crane,US,31.3974,-102.3501,24,
455,ad dilam,SA,23.9915,47.1627,17,
462,san miguel,SV,13.4833,-88.1833,49,
476,lachi,PK,33.3833,71.3386,40,


In [49]:
hotel_map = hotel_df.hvplot.points(
            'Lng',
            'Lat',
            geo = True,
            tiles = 'OSM',
            size = 'Humidity',
            frame_width = 700,
            frame_height = 500,
            color = 'City',
            alpha = 0.5)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [93]:
# Set parameters to search for a hotel
radius = 10000


params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    long = hotel_df.loc[index, "Lng"]
    
    # Alternatively I can use
    #lat = row["Lat"]
    #long = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)


    # Convert the API response to JSON format
    name_address = name_address.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
gadani - nearest hotel: Al Khalid
bakewar - nearest hotel: Leela Palace
meadow lake - nearest hotel: Los Lunas Inn & Suites
fort stockton - nearest hotel: La Quinta Inn
crane - nearest hotel: Mainstay Suites
ad dilam - nearest hotel: شقق فندقية
san miguel - nearest hotel: Continental Plaza Hotel
lachi - nearest hotel: Marriot hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
79,gadani,PK,25.1185,66.7299,81,Al Khalid
91,bakewar,IN,26.6667,79.1833,60,Leela Palace
350,meadow lake,US,34.8014,-106.5436,21,Los Lunas Inn & Suites
393,fort stockton,US,30.8940,-102.8793,24,La Quinta Inn
421,crane,US,31.3974,-102.3501,24,Mainstay Suites
455,ad dilam,SA,23.9915,47.1627,17,شقق فندقية
462,san miguel,SV,13.4833,-88.1833,49,Continental Plaza Hotel
476,lachi,PK,33.3833,71.3386,40,Marriot hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [90]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points(
            'Lng',
            'Lat',
            geo = True,
            tiles = 'OSM',
            size = 'Humidity',
            frame_width = 700,
            frame_height = 500,
            color = 'City',
            alpha = 0.5,
            hover_cols=["Hotel Name", "Country"]
            )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)